In [1]:
cd notebooks/

[Errno 2] No such file or directory: 'notebooks/'
/home/nicit/deep_learning_course_wfco/notebooks


In [2]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import torch

import sys
sys.path.append("../src")
import utils

In [3]:
folder_path = "/home/nicit/nordsoen/nordsoen-i-wflo/results/DL/recorders_SLSQP_simple/"

recorders = {}
n_layouts = 1000
# load pickle
for random_state in trange(int(n_layouts/10), desc="Loading recorders"):
    with open(folder_path + f"recorder_{random_state}.pkl", "rb") as f:
                recorders[random_state] = pickle.load(f)

Loading recorders:   0%|          | 0/100 [00:00<?, ?it/s]

Loading recorders: 100%|██████████| 100/100 [00:21<00:00,  4.58it/s]


In [4]:
recorders[5][6000].keys()

dict_keys(['yaw', 'x', 'y', 'x_rot', 'y_rot', 'd_x', 'd_y', 'sort_idx', 'ws_eff', 'ti_eff', 'ws', 'aep', 'aep_opt'])

In [5]:
print('x_rot shape: ', recorders[5][6000]['x_rot'].shape)
print('y_rot shape: ', recorders[5][6000]['y_rot'].shape)
print('yaw shape: ', recorders[5][6000]['yaw'].shape)
print('ws_eff shape: ', recorders[5][6000]['ws_eff'].shape)
print('ti_eff shape: ', recorders[5][6000]['ti_eff'].shape)
print('ws shape: ', recorders[5][6000]['ws'].shape)
print('aep shape: ', recorders[5][6000]['aep'].shape)
print('aep_opt shape: ', recorders[5][6000]['aep_opt'].shape)
print('yaw shape: ', recorders[5][6000]['yaw'].shape)

x_rot shape:  (31, 5)
y_rot shape:  (31, 5)
yaw shape:  (5, 31, 9)
ws_eff shape:  (5, 31, 9)
ti_eff shape:  (5, 31, 9)
ws shape:  (9,)
aep shape:  (31, 9)
aep_opt shape:  (31, 9)
yaw shape:  (5, 31, 9)


In [6]:
n_layouts = 1000
rotor_diameter = 284.0  # adjust to your turbine specs
nearest_idx = [0, 1, 2, 3, 4]


# --- Step 1: Load and aggregate all layouts ---
all_x_rot = []
all_y_rot = []
all_ws_eff = []
all_ti_eff = []
all_yaw = []
all_ws = []

for random_state in tqdm(range(int(n_layouts / 10)), desc="Loading recorders"):

    file_path = folder_path + f"recorder_{random_state}.pkl"
    with open(file_path, "rb") as f:
        recorder = pickle.load(f)
    for result in recorder.values():
        all_x_rot.append(result["x_rot"])       # (n_dir, n_turbines)
        all_y_rot.append(result["y_rot"])
        all_ws_eff.append(result["ws_eff"])     # (n_turbines, n_dir, n_ws)
        all_ti_eff.append(result["ti_eff"])
        all_yaw.append(result["yaw"])
        all_ws.append(result["ws"])             # (n_ws,)

# Stack everything
x_rot_all = np.stack(all_x_rot)       # (n_layouts, n_dir, n_turbines)
y_rot_all = np.stack(all_y_rot)
ws_eff_all = np.stack(all_ws_eff)     # (n_layouts, n_turbines, n_dir, n_ws)
ti_eff_all = np.stack(all_ti_eff)
yaw_all = np.stack(all_yaw)
ws_all = np.stack(all_ws)             # (n_layouts, n_ws)

n_layouts_total, n_dir, n_turbines = x_rot_all.shape
n_ws = ws_all.shape[1]
n_cases = n_layouts_total * n_dir * n_ws

# print(f"Aggregated shapes: x_rot_all={x_rot_all.shape}, ws_eff_all={ws_eff_all.shape}")

# --- Step 2: Compute dx, dy for all layouts and directions ---
# Flatten layouts and directions for _process_layout
x_rot_flat = x_rot_all.reshape(-1, n_turbines)  # (n_layouts * n_dir, n_turbines)
y_rot_flat = y_rot_all.reshape(-1, n_turbines)

dx_all, dy_all = utils._process_layout(
    turbine_x=x_rot_flat,
    turbine_y=y_rot_flat,
    rotor_diameter=rotor_diameter,
    nearest_idx=nearest_idx,
    normalize=True,
    spread=.1
)  # shape: (len(nearest_idx), n_turbines, n_layouts * n_dir)

# Repeat across wind speeds
dx_flat = np.repeat(dx_all, n_ws, axis=2).reshape(len(nearest_idx), n_turbines, n_cases)
dy_flat = np.repeat(dy_all, n_ws, axis=2).reshape(len(nearest_idx), n_turbines, n_cases)

# --- Step 3: Flatten dynamic inputs ---
ws_eff_flat = ws_eff_all.reshape(n_layouts_total, n_turbines, n_dir * n_ws)
ws_eff_flat = ws_eff_flat.transpose(1, 0, 2).reshape(n_turbines, n_cases)

ti_eff_flat = ti_eff_all.reshape(n_layouts_total, n_turbines, n_dir * n_ws)
ti_eff_flat = ti_eff_flat.transpose(1, 0, 2).reshape(n_turbines, n_cases)

yaw_flat = yaw_all.reshape(n_layouts_total, n_turbines, n_dir * n_ws)
yaw_flat = yaw_flat.transpose(1, 0, 2).reshape(n_turbines, n_cases)

# Repeat ws for all layouts and directions
# ws_rep = np.tile(ws_all.reshape(-1), n_dir)  # (n_cases,)
ws_rep = np.tile(ws_all.reshape(n_layouts_total, n_ws), (1, n_dir)).reshape(n_cases)

# --- Step 4: Build feature matrix in one shot ---
dx_part = dx_flat.transpose(1, 2, 0)  # (n_turbines, n_cases, 5)
dy_part = dy_flat.transpose(1, 2, 0)

# modify this so that it is -1 when it is not waked (or waked), instaed of zero!
waked_part = np.where(dx_part != np.inf, 1.0, -1.0)

# Replace inf with nan for dx/dy
dx_part[dx_part == np.inf] = np.nan

# when there is nan in the corresponding dx, set dy to nan as well
dy_part[dy_part == np.inf] = np.nan

# Ensure dy is NaN wherever dx is NaN
dy_part[np.isnan(dx_part)] = np.nan

# adjust so that whenever the ws_eff is >= 12, the yaw should be zero
yaw_flat = np.where((ws_eff_flat >= 10.5) 
                    # & (np.abs(yaw_flat) == 25.0)
                    , 0.0, yaw_flat)

# same whenever ws_eff is < 4
yaw_flat = np.where(ws_eff_flat < 4.0, 0.0, yaw_flat)

# round the yaw to the nearest float with 1 decimal place
yaw_flat = np.round(yaw_flat, 1)

# Static features: dx, dy, waked flags
static_features = np.concatenate([dx_part, dy_part, 
                                waked_part
                                  ], axis=2)  # (n_turbines, n_cases, 15)

# merge ws and ws_eff into one feature (the difference)
dynamic_features = np.stack([
    ws_eff_flat,
    ti_eff_flat,
    np.tile(ws_rep, (n_turbines, 1))
], axis=2)

# Combine all features
X = np.concatenate([static_features, dynamic_features], axis=2)  # (n_turbines, n_cases, 18)
X = X.reshape(-1, X.shape[2])  # (n_turbines * n_cases, 18)
Y = yaw_flat.reshape(-1)

# Convert to torch tensors
X_torch = torch.tensor(X, dtype=torch.float32)
Y_torch = torch.tensor(Y, dtype=torch.float32)

print(f"Final dataset shapes: X = {X_torch.shape}, Y = {Y_torch.shape}")

# Save dataset
torch.save((X_torch, Y_torch), f"../data/5wt_dataset_{n_layouts}_slsqp_simple_complete.pt")
print(f"Dataset saved to 5wt_dataset_{n_layouts}_slsqp_simple_complete.pt")


Loading recorders: 100%|██████████| 100/100 [00:07<00:00, 13.60it/s]


Final dataset shapes: X = torch.Size([133920000, 18]), Y = torch.Size([133920000])
Dataset saved to 5wt_dataset_1000_slsqp_simple_complete.pt


In [7]:
# check nans in X
print('Number of NaNs in X: ', torch.isnan(X_torch).sum().item())
print('Shape of X: ', X_torch.shape)
print("Number of total data points in X", X_torch.shape[0]*X_torch.shape[1])
print("Percentage of NaNs in X", round(torch.isnan(X_torch).sum().item()/(X_torch.shape[0]*X_torch.shape[1])*100, 2), "%")

RuntimeError: [enforce fail at alloc_cpu.cpp:118] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 19284480000 bytes. Error code 12 (Cannot allocate memory)

In [8]:
# i 10 is the weird stuff
i = 30000
print(X_torch[i])
print(Y_torch[i])

tensor([ 1.5108e+01,  2.2699e+01,  2.8025e+01,         nan,         nan,
        -6.5428e-01,  2.5221e-02,  1.4618e+00,         nan,         nan,
         1.0000e+00,  1.0000e+00,  1.0000e+00, -1.0000e+00, -1.0000e+00,
         7.0000e+00,  6.0000e-02,  7.0000e+00])
tensor(-4.1000)


In [9]:
# get indeces where Y is 25 or -25
weird_indices = torch.where((Y_torch == 25.0) | (Y_torch == -25.0))[0]
print("Weird indices where Y is 25 or -25: ", weird_indices)
# counte them
print("Number of weird indices: ", len(weird_indices))
for idx in weird_indices:
    print(f"Index: {idx.item()}, X: {X_torch[idx, 15]}, Y: {Y_torch[idx]}")

Weird indices where Y is 25 or -25:  tensor([    92517,    272094,   1111164,   1717728,   2001939,   2099061,
          2168565,   2359311,   2633712,   2740245,   2959143,   3420447,
          3538392,   4118130,   4159176,   4211649,   4297551,   4440228,
          4589457,   5113887,   5569152,   5878679,   5878680,   6369378,
          6570852,   6875250,   7286837,   8079198,   8655909,   8866824,
          9033018,   9096117,   9224502,   9474450,   9570993,   9586383,
          9610098,   9639816,   9653856,  10107771,  10950369,  11033250,
         11285717,  11396130,  11699133,  11835051,  11854248,  11872157,
         12863111,  13015968,  13390494,  13660782,  13954623,  14013951,
         14340183,  14408790,  14539398,  14927226,  15257291,  15298890,
         15616077,  15631989,  15744435,  15819414,  15921537,  16131363,
         16346175,  16416141,  16911429,  17141055,  17204940,  17415033,
         17567349,  18538458,  18724686,  18905442,  19117266,  19274298,
 

In [14]:
idx = 80977901
print(X_torch[idx])
Y_torch[idx]

tensor([ 5.3493,     nan,     nan,     nan,     nan, -0.1519,     nan,     nan,
            nan,     nan,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  8.7829,
         0.0703,  9.0000])


tensor(-25.)

: 

In [11]:
# get index where last value is < 3.2
indices = (X_torch[:, -1] < 3.2).nonzero(as_tuple=True)[0]
print(f"Number of samples with wind speed < 3.2 m/s: {len(indices)}")
indices[0]

Number of samples with wind speed < 3.2 m/s: 0


IndexError: index 0 is out of bounds for dimension 0 with size 0

: 

In [ ]:
# get index where last value is < 3.2
indices = (X_torch[:, -1] < 3.2).nonzero(as_tuple=True)[0]
print(f"Number of samples with wind speed < 3.2 m/s: {len(indices)}")
indices[0]